# Carrier Structured Ingestion
Process invoices, tracking, rating outputs and compute variance.

In [ ]:
import pandas as pd
from datetime import datetime

invoices = pd.read_csv('landing/carrier/invoices.csv')
rating = pd.read_csv('landing/carrier/rating.csv')

invoices['ShipDate'] = pd.to_datetime(invoices['ShipDate'])
rating['RatedAmount'] = rating['RatedAmount'].astype(float)

variance = invoices.merge(rating, on='InvoiceLineId', how='left')
variance['Variance'] = variance['BilledAmount'] - variance['RatedAmount']
variance['Reason'] = variance['Reason'].fillna('')

invoices.to_parquet('curated/FactInvoice.parquet', index=False)
variance[['InvoiceLineId','InvoiceId','Carrier','ServiceLevel','SKU','BilledAmount','RatedAmount','Variance','Reason']].to_parquet('curated/RatingOutput.parquet', index=False)
print('Wrote curated Carrier tables at', datetime.utcnow(), 'UTC')
